In [1]:
import torch
import random
import numpy as np
from functools import partial

In [2]:
# pip install torch torch_geometric torch-scatter pandas scikit-learn wandb dill openpyxl torchmetrics
# python -m pip install gurobipy

In [3]:
from temp.data.gen import parallel_generate_problem, parallel_generate_solutions
from temp.data.problem import setcover, lot_sizing
from temp.data.info import ModelInfo
from temp.data.dataset import ModelGraphDataset
from temp.data.aug import parallel_augment_info, augment_info


from temp.solver.utils import solve_inst, with_lic


In [4]:
PRE_TRAIN_DIR = "temp/pre_train_tiny"
# TRAIN_DIR = "temp/train_ls"
# VALID_DIR = "temp/valid_ls"

In [5]:
import json
with open("gb.lic") as f:
    lic = json.load(f)

# parallel_generate_problem(lot_sizing, PRE_TRAIN_DIR, 100, 10)
# parallel_generate_solutions(PRE_TRAIN_DIR, 5, lic=lic)

# parallel_generate_problem(lot_sizing, TRAIN_DIR, 10000, 10)
# parallel_generate_solutions(TRAIN_DIR, 5, lic=lic)

# parallel_generate_problem(lot_sizing, VALID_DIR, 100, 10)
# parallel_generate_solutions(VALID_DIR, 5, lic=lic)
# assert 1 == 2

Restricted license - for non-production use only - expires 2026-11-23
Restricted license - for non-production use only - expires 2026-11-23
Restricted license - for non-production use only - expires 2026-11-23
Restricted license - for non-production use only - expires 2026-11-23
Restricted license - for non-production use only - expires 2026-11-23
Restricted license - for non-production use only - expires 2026-11-23
Restricted license - for non-production use only - expires 2026-11-23
Restricted license - for non-production use only - expires 2026-11-23
Restricted license - for non-production use only - expires 2026-11-23
Restricted license - for non-production use only - expires 2026-11-23
Set parameter CloudAccessID
Set parameter CloudSecretKey
Set parameter CloudPool to value "831775-C3Dev"
Set parameter CSAppName to value "Josh"
Set parameter CloudAccessID
Set parameter CloudSecretKey
Set parameter CloudPool to value "831775-C3Dev"
Set parameter CSAppName to value "Josh"
Set parame

In [6]:
# import os
# import shutil

# # Define the source and destination folders
# src_folder = "temp/valid_ls"
# dest_folder = "temp/valid_ls_backup"

# # Create the destination folder if it doesn't exist
# os.makedirs(dest_folder, exist_ok=True)

# # List all files in the source folder
# files = os.listdir(src_folder)

# # First, process all .lp files: if a corresponding .npz file exists, move both to "done"
# for file_name in os.listdir(src_folder):
#     if file_name.endswith(".lp"):
#         lp_path = os.path.join(src_folder, file_name)
#         base_name = os.path.splitext(file_name)[0]
#         npz_file = base_name + ".npz"
#         npz_path = os.path.join(src_folder, npz_file)
        
#         # If the corresponding .npz exists, move both files
#         if os.path.exists(npz_path):
#             shutil.move(lp_path, os.path.join(dest_folder, file_name))
#             shutil.move(npz_path, os.path.join(dest_folder, npz_file))
#             print(f"Moved pair: {file_name} and {npz_file}")

# # Next, remove any .npz files that are left without a corresponding .lp file.
# for file_name in os.listdir(src_folder):
#     if file_name.endswith(".npz"):
#         base_name = os.path.splitext(file_name)[0]
#         lp_file = base_name + ".lp"
#         lp_path = os.path.join(src_folder, lp_file)
#         npz_path = os.path.join(src_folder, file_name)
        
#         if not os.path.exists(lp_path):
#             os.remove(npz_path)
#             print(f"Removed orphan npz: {file_name}")

In [7]:
# m = lot_sizing()
# model = m

# # model.setParam("OutputFlag", 0)
# model.setParam("PoolSolutions", 10)
# model.setParam("PoolSearchMode", 2)
# model = with_lic(model)
# model.optimize()

# vs = model.getVars()
# obj_val_and_sols = []

# for i in range(model.SolCount):
#     # TODO: setting solution number actually takes long time try to optimize
#     # TODO: add function to round by tolerance
#     model.params.SolutionNumber = i
#     obj_val = model.PoolObjVal
#     s = [obj_val] + [v.Xn for v in vs]
#     obj_val_and_sols.append(s)
# model.dispose()

# info = ModelInfo.from_model(m)
# info.var_info.sols = np.array(obj_val_and_sols)

In [8]:
# from temp.data.dataset import info_to_data
# d = info_to_data(info)
# aug = partial(parallel_augment_info, n=100)

In [9]:
# import os
# import pickle
# from temp.data.aug import augment_info, get_constraint_violations, get_lhs_matrix

# for p in os.listdir(os.path.join(PRE_TRAIN_DIR, "processed")):
#     if not p.endswith("pkl"):
#         continue
#     with open(os.path.join(PRE_TRAIN_DIR, "processed", p), "rb") as f:
#         info = pickle.load(f)

#     lhs = get_lhs_matrix(info.var_info.n, info.con_info)
#     rhs = np.array(info.con_info.rhs)
#     ops = np.array(info.con_info.types)
#     violations = get_constraint_violations(lhs, info.var_info.sols[0, 1:], rhs, ops)
#     assert not (np.abs(violations) >= 1e-5).any(), violations

    
#     # for _ in range(100):
#     #     aug_info = info.copy()
#     #     augment_info(aug_info)
#     #     lhs = get_lhs_matrix(aug_info.var_info.n, aug_info.con_info)
#     #     rhs = np.array(aug_info.con_info.rhs)
#     #     ops = np.array(aug_info.con_info.types)
#     #     violations = get_constraint_violations(lhs, aug_info.var_info.sols[0, 1:], rhs, ops)
#     #     assert not (np.abs(violations) >= 1e-5).any(), violations

In [10]:
from temp.data.graph import get_bipartite_graph, add_label
from temp.data.dataset import create_data_object

In [11]:
pre_train_d = ModelGraphDataset(PRE_TRAIN_DIR, augment=augment_info, force_reload=True)
train_d = ModelGraphDataset(PRE_TRAIN_DIR, augment=augment_info)
valid_d = ModelGraphDataset(PRE_TRAIN_DIR)

for i in range(len(pre_train_d)):
    data = pre_train_d[i][1]

Processing...
(model, solution) => info:   0%|          | 0/100 [00:00<?, ?it/s]

(model, solution) => info: 100%|██████████| 100/100 [00:11<00:00,  8.35it/s]
Done!


>>> tensor(1.0000e+10)
>>> tensor(1.0000e+10)
>>> tensor(1.0000e+10)
>>> tensor(1.0000e+10)
>>> tensor(1.0000e+10)
>>> tensor(1.0000e+10)
>>> tensor(1.0000e+10)
>>> tensor(1.0000e+10)
>>> tensor(1.0000e+10)
>>> tensor(1.0000e+10)
>>> tensor(1.0000e+10)
>>> tensor(1.0000e+10)
>>> tensor(1.0000e+10)
>>> tensor(1.0000e+10)
>>> tensor(1.0000e+10)
>>> tensor(1.0000e+10)
>>> tensor(1.0000e+10)
>>> tensor(1.0000e+10)
>>> tensor(1.0000e+10)
>>> tensor(1.0000e+10)


In [12]:
var_feature_size = data.var_node_features.size(-1)
con_feature_size = data.con_node_features.size(-1) 

In [13]:
import pandas as pd
CFG = pd.read_excel("temp/trained_models/setcover_model_configs.xlsx", index_col=0).loc[0].to_dict()
CFG["num_epochs"] = 32
CFG["num_layers"] = 16
CFG["hidden"] = 32

In [14]:
from temp.model.utils import get_model
from temp.model.trainer import train

In [15]:
model_name, model, criterion, optimizer, scheduler = get_model(".", var_feature_size, con_feature_size, n_batches=1, **CFG)

In [16]:
scheduler.total_steps = 10000

In [17]:
def seed_worker(worker_id):
    worker_seed = torch.initial_seed()
    np.random.seed(worker_seed)
    random.seed(worker_seed)

from torch_geometric.loader import DataLoader
pretrain_loader = DataLoader(pre_train_d, batch_size=2, shuffle=True, generator=torch.Generator().manual_seed(0))
train_loader = DataLoader(train_d, batch_size=2, shuffle=True, num_workers=1, generator=torch.Generator().manual_seed(0))
val_loader = DataLoader(valid_d, batch_size=2, shuffle=True, generator=torch.Generator().manual_seed(0))

In [18]:
scheduler.total_steps = 1000000

In [ ]:
train(model_name, model, criterion, optimizer, scheduler, pretrain_loader, train_loader, val_loader, CFG, False, "./")

>> Training starts on the current device cpu
>> Pretraining for prenorm...


  4%|▍         | 2/50 [00:00<00:17,  2.69it/s]

>>> tensor(1.0000e+10)


 18%|█▊        | 9/50 [00:03<00:17,  2.34it/s]

>>> tensor(1.0000e+10)


...

>>> tensor(1.0000e+10)


 66%|██████▌   | 33/50 [00:11<00:06,  2.59it/s]

>>> tensor(1.0000e+10)


 82%|████████▏ | 41/50 [00:12<00:01,  6.17it/s]

>>> tensor(1.0000e+10)


In [20]:
assert 1 == 2

AssertionError: 

In [21]:
from temp.data.dataset import info_to_data

In [45]:
m = setcover(256, 256)

In [46]:
info = ModelInfo.from_model(m)
data = info_to_data(info)

In [47]:
with torch.no_grad():
    logits = model(data)

In [48]:
binary_mask = torch.ones(len(logits), dtype=bool)

In [49]:
from temp.solver import upr

In [50]:
probs, preds = upr.get_predictions(logits, binary_mask)

In [51]:
unc = upr.get_uncertainty(logits)

In [52]:
threshold = upr.get_threshold(unc)

In [53]:
import json
from tempfile import NamedTemporaryFile

import gurobipy as gp


def fix_var(inst, idxs, vals):
    assert len(idxs) == len(vals)
    bounds = {}
    vs = inst.getVars()
    for idx, val in zip(idxs, vals):
        v = vs[idx]
        bounds[idx] = (v.lb, v.ub)
        v.setAttr("lb", val)
        v.setAttr("ub", val)
    inst.update()
    return bounds


def unfix_var(inst, idxs, bounds):
    assert len(idxs) == len(bounds)
    vs = inst.getVars()
    print(idxs, bounds)
    for i, (lb, ub) in zip(idxs, bounds):
        vs[i].setAttr("lb", lb)
        vs[i].setAttr("ub", ub)


def get_iis_vars(inst):
    try:
        inst.computeIIS()
    except Exception as e:
        print(e)
        if "Cannot compute IIS on a feasible model" in str(e):
            return set()
        raise e

    with NamedTemporaryFile(suffix=".ilp", mode="w+") as f:
        inst.write(f.name)
        f.seek(0)
        return set(f.read().split())


def set_starts(inst, starts):
    vs = inst.getVars()
    for i, s in starts.items():
        vs[i].setAttr("lb", s)


def solve_inst(inst):
    vs = inst.getVars()
    inst.optimize()
    return inst.getAttr("X", vs)


def repair(inst, fixed: set, bounds: dict):
    old_iis_method = getattr(inst, "IISMethod", -1)
    inst.setParam("IISMethod", 0)

    vs = inst.getVars()
    ns = inst.getAttr("varName", vs)
    name_to_idx = {n: i for i, n in enumerate(ns)}

    freed = set()
    while iis_var_names := get_iis_vars(inst):
        for n in iis_var_names:
            if n not in name_to_idx:
                continue

            var_idx = name_to_idx[n]
            if var_idx not in fixed:
                continue

            if var_idx in freed:
                continue

            lb, ub = bounds[var_idx]
            vs[var_idx].lb = lb
            vs[var_idx].ub = ub
            freed.add(var_idx)

    inst.setParam("IISMethod", old_iis_method)
    return freed


def with_lic(m, path="gb.lic"):
    with open(path) as f:
        env = gp.Env(params=json.load(f))
    return m.copy(env=env)


In [54]:
import numpy as np
import torch
import torch.nn.functional as F

# from temp.solver.utils import fix_var, repair, set_starts, unfix_var, solve_inst

EVIDENCE_FUNCS = {
    "softplus": (lambda y: F.softplus(y)),
    "relu": (lambda y: F.relu(y)),
    "exp": (lambda y: torch.exp(torch.clamp(y, -10, 10))),
}


def get_predictions(logits, binary_mask):
    probs = torch.softmax(logits, axis=1)
    preds = probs[:, 1]
    probs = _to_numpy(probs)
    preds = _to_numpy(preds).squeeze()
    preds[binary_mask] = preds[binary_mask].round()
    return probs, preds


def get_uncertainty(logits, evidence_func_name: str = "softplus"):
    evidence = EVIDENCE_FUNCS[evidence_func_name](logits)
    alpha = evidence + 1
    uncertainty = logits.shape[1] / torch.sum(alpha, dim=1, keepdim=True)
    return uncertainty


def get_threshold(uncertainty: torch.Tensor, r_min: float = 0.4, r_max: float = 0.55):
    q = (r_min + r_max) / 2
    threshold = torch.quantile(uncertainty, q)
    r = (uncertainty <= threshold).float().mean()

    if r > r_max:
        threshold = torch.quantile(uncertainty, r_max)
        return threshold

    if r < r_min:
        threshold = torch.quantile(uncertainty, r_min)
        return threshold

    return threshold


def get_confident_idx(indices, uncertainty, threshold):
    confident_mask = uncertainty <= threshold
    confident_idx = indices[confident_mask]
    return confident_idx.sort()[0]


def solve(inst, prediction, uncertainty, indices, max_iter):
    threshold = get_threshold(uncertainty)
    conf_idxs = get_confident_idx(indices, uncertainty, threshold)
    conf_vals = prediction[conf_idxs]
    bounds = fix_var(inst, conf_idxs, conf_vals)
    print(len(bounds), len(prediction))

    min_q = sum(uncertainty <= threshold) / len(uncertainty)
    max_q = 1.0
    dq = (max_q - min_q) / (max_iter - 1)

    fixed = set(conf_idxs.tolist())
    freed = set(repair(inst, fixed, bounds))
    for i in range(1, max_iter):
        sol = solve_inst(inst)
        q = max_q - dq * i
        threshold = np.quantile(uncertainty, q)
        conf_idxs = get_confident_idx(indices, uncertainty, threshold)
        to_unfix = list(fixed - set(conf_idxs.tolist()))
        to_unfix = [i for i in to_unfix if i not in freed]
        unfix_var(inst, to_unfix, [bounds[i] for i in to_unfix])
        starts = {i: sol[i] for i in to_unfix}
        starts.update({i: sol[i] for i in freed})
        set_starts(inst, starts)
    return sol


def _to_numpy(tensor_obj):
    return tensor_obj.cpu().detach().numpy()


In [55]:
indices = torch.arange(0, len(unc), dtype=int)
unc = unc.squeeze()
to_solve = m.copy()
to_solve.setParam("TimeLimit", 40)
to_solve.setParam("NoRelHeurTime", 20)
solve(to_solve, preds, unc, indices, 5)

Set parameter TimeLimit to value 40
Set parameter NoRelHeurTime to value 20
122 256
Set parameter IISMethod to value 0
Gurobi Optimizer version 12.0.1 build v12.0.1rc0 (mac64[x86] - Darwin 22.4.0 22E252)

CPU model: Intel(R) Core(TM) i9-9980HK CPU @ 2.40GHz
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Non-default parameters:
TimeLimit  40
NoRelHeurTime  20
IISMethod  0

IIS computation: initial model status unknown, solving to determine model status
Found heuristic solution: objective 0.0000000

Explored 0 nodes (0 simplex iterations) in 0.00 seconds (0.00 work units)
Thread count was 1 (of 16 available processors)

Solution count 1: 0 

Optimal solution found (tolerance 1.00e-04)
Best objective 0.000000000000e+00, best bound 0.000000000000e+00, gap 0.0000%
IIS runtime: 0.01 seconds (0.00 work units)
Cannot compute IIS on a feasible model
Set parameter IISMethod to value -1
Gurobi Optimizer version 12.0.1 build v12.0.1rc0 (mac64[x86] - Darwin 22.4.0 22

[0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 1.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 1.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 1.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 1.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 1.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 1.0,
 0.0,
 1.0,
 0.0,
 0.0,
 1.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 1.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 1.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 1.0,
 0.0,
 0.0,
 1.0,
 0.0,
 0.0,
 0.0,
 0.0,
 1.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 1.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 1.0,
 0.0,
 0.0,
 0.0,
 1.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0

In [56]:
gb_m = m.copy()
gb_m.setParam("TimeLimit", 150)
gb_m.setParam("NoRelHeurTime", 60)
gb_m.optimize()

Set parameter TimeLimit to value 150
Set parameter NoRelHeurTime to value 60
Gurobi Optimizer version 12.0.1 build v12.0.1rc0 (mac64[x86] - Darwin 22.4.0 22E252)

CPU model: Intel(R) Core(TM) i9-9980HK CPU @ 2.40GHz
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Non-default parameters:
TimeLimit  150
NoRelHeurTime  60

Optimize a model with 256 rows, 256 columns and 6529 nonzeros
Model fingerprint: 0x6cdcc514
Variable types: 0 continuous, 256 integer (256 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [5e+00, 2e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Found heuristic solution: objective 298.0000000
Presolve time: 0.01s
Presolved: 256 rows, 256 columns, 6529 nonzeros
Variable types: 0 continuous, 256 integer (256 binary)
Starting NoRel heuristic
Found heuristic solution: objective 245.0000000
Found heuristic solution: objective 239.0000000
Found heuristic solution: objective 192.0000000